In [1]:
import json
from collections import Counter, defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import StratifiedKFold


In [2]:
DATA_DIR = "/opt/ml/detection/dataset"

# Split data

- reference
https://www.kaggle.com/backtracking/smart-data-split-train-eval-for-object-detection/notebook

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=100)
ax.bar(x=[_ for _ in range(1, 8)], height=category_per_image_dict.values())
for key in category_per_image_dict:
    ax.text(key, category_per_image_dict[key]+100, category_per_image_dict[key], ha='center', fontdict={'size': 12})
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_title('Categories per image', fontdict={'size': 13})
ax.set_xlabel('Number of categories', fontdict={'size': 12})

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=100)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 0, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 1, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 2, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 3, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 4, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 5, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 6, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 7, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 8, 'area'], ax=ax, log_scale=True)
sns.kdeplot(annotations_df.loc[annotations_df['category_id'] == 9, 'area'], ax=ax, log_scale=True)
ax.set_xlim(1e2, 1.5e6)

# Make JSON format

In [3]:
file_name = f"{DATA_DIR}/train.json"

class ReadCOCOjson:
    def __init__(self, json_file: str) -> None:
        with open(json_file, 'r') as f:
            self.json_dict = json.load(f) 
        # print(f"json_key: {self.json_dict.keys()}")
        self.images_df = pd.DataFrame(self.json_dict['images'])
        self.annotations_df = pd.DataFrame(self.json_dict['annotations'])
    
    def get_extra_coord(self):
        bbox_array = np.array(self.annotations_df['bbox'].to_list())
        # x
        self.annotations_df['x_min'] = bbox_array[:, 0]
        self.annotations_df['x_center'] = bbox_array[:, 0] + bbox_array[:, 2]/2
        self.annotations_df['x_max'] = bbox_array[:, 0] + bbox_array[:, 2]
        # y
        self.annotations_df['y_min'] = bbox_array[:, 1]
        self.annotations_df['y_center'] = bbox_array[:, 1] + bbox_array[:, 3]/2
        self.annotations_df['y_max'] = bbox_array[:, 1] + bbox_array[:, 3]
        # w
        self.annotations_df['w'] = bbox_array[:, 2]
        # h
        self.annotations_df['h'] = bbox_array[:, 3]

coco_json = ReadCOCOjson(file_name)

In [ ]:
coco_json.json_dict['categories']

In [4]:
coco_json.get_extra_coord()
df = coco_json.annotations_df.copy()

In [ ]:
train_df = pd.read_csv(f"{DATA_DIR}/annotations_train.csv", converters={'bbox': eval})
valid_df = pd.read_csv(f"{DATA_DIR}/annotations_valid.csv", converters={'bbox': eval})

In [38]:
with open(f'{DATA_DIR}/train.json', 'r') as f:
    origin_json = json.load(f)

origin_images_df =  pd.DataFrame(origin_json['images'])

dummy_json = origin_json.copy()
dummy_json['images'] = []
dummy_json['annotations'] = []

train_json = dummy_json.copy()
valid_json = dummy_json.copy()

# train
## annotations
json_result = train_df.loc[:, ['image_id', 'category_id', 'area', 'bbox', 'iscrowd', 'id']].to_json(orient="table", index=False)
parsed = json.loads(json_result)
train_json['annotations'] = parsed['data']
## origin_json images
json_result = origin_images_df.loc[train_df['image_id'].unique()].to_json(orient="table", index=False)
parsed = json.loads(json_result)
train_json['images'] = parsed['data']

# valid
## annotations
json_result = valid_df.loc[:, ['image_id', 'category_id', 'area', 'bbox', 'iscrowd', 'id']].to_json(orient="table", index=False)
parsed = json.loads(json_result)
valid_json['annotations'] = parsed['data']
## origin_json images
json_result = origin_images_df.loc[valid_df['image_id'].unique()].to_json(orient="table", index=False)
parsed = json.loads(json_result)
valid_json['images'] = parsed['data']

# Save json file format
with open('stratified_train.10fold.wArea.json', 'w') as f:
    json.dump(train_json, f, indent=2)
with open('stratified_valid.10fold.wArea.json', 'w') as f:
    json.dump(valid_json, f, indent=2)

In [ ]:
df.head()

In [ ]:
import random

In [31]:
# x = df['id']
# y = df['category_id']
# groups = df['image_id']
seed = 123456
k = 10

In [ ]:
%%timeit
df.groupby(['image_id', 'category_id']).size().unstack(fill_value=0)
# y.value_counts().to_dict()
y.value_counts()

In [ ]:
%%timeit
y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
y_distr = Counter()
for label, g in zip(y, groups):
    y_counts_per_group[g][label] += 1
    y_distr[label] += 1

In [ ]:
df.groupby(['image_id', 'category_id']).size().unstack(fill_value=0)

In [ ]:
# %%timeit
# 7.23 s ± 141 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
y = df['category_id']
groups = df['image_id']

labels_num = np.max(y) + 1  # label이 0~9로 구성이 되어 있어 최대 값에 1을 더해 label 개수를 파악합니다.
# 각 image 별로 label 개수를 확인하기 위해 지정합니다.
y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))  # dict shape: (n_image, n_label)
# 각 label 별 개수를 확인하기 위해 지정합니다.
y_distr = Counter()
# annotation 데이터를 이용해 순차적으로 데이터를 확인합니다.
for label, g in zip(y, groups):
    y_counts_per_group[g][label] += 1
    y_distr[label] += 1

y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))  # fold별 각 label 개수를 파악하기 위해 지정합니다.

# 어떤 image가 특정 fold에 속할 때 전체 fold의 std 값 확인하는 함수입니다.
def eval_y_counts_per_fold(y_counts, fold):
    y_counts_per_fold[fold] += y_counts  # image가 특정 fold에 속한 것을 반영합니다.
    std_per_label = []
    # label 별로 전체 fold에 대해 std 값을 확인합니다.
    for label in range(labels_num):
        # label 별로 개수가 다르기 때문에 scale을 맞추기 위해 각 label 별 총 개수로 나누어 줍니다.
        label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
        std_per_label.append(label_std)
    # 본 함수는 해당 image가 특정 fold에 속한 것을 가정하고 계산합니다. 해당 image가 특정 fold에 속하지 않을 수 있기 때문에 원상 복구 합니다.
    y_counts_per_fold[fold] -= y_counts
    # label별 std를 평균냅니다.
    return np.mea가(std_per_label)

# image당 각 label 개수에 대한 값들만 활용합니다.
groups_and_y_counts = list(y_counts_per_group.items())
# shuffle 후 sort 하는 것이 이상해 보이지만, sort 하였을 때 같은 std 값일 경우 순서가 달라지게 됩니다.
random.Random(seed).shuffle(groups_and_y_counts)
groups_per_fold = defaultdict(set)  # 각 fold별 index를 저장하기 위해 지정합니다.

# 큰 std를 가진 image부터 순차적으로 best fold를 찾습니다.
for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
    best_fold = None
    min_eval = None
    # image가 어떤 fold에 속할 때 가장 낮은 std를 가지는지 확인하는 과정입니다.
    for i in range(k):  # 각 fold에 대해 순차적으로 확인합니다.
        fold_eval = eval_y_counts_per_fold(y_counts, i)  # 전체 fold에 대해 std값 확인.
        # std 값이 가장 낮을 때가 best fold 입니다.
        if min_eval is None or fold_eval < min_eval:
            min_eval = fold_eval
            best_fold = i
    y_counts_per_fold[best_fold] += y_counts  # best fold를 확정하고 해당 image를 반영합니다.
    groups_per_fold[best_fold].add(g)  # best fold에 해당 이미지의 index를 추가합니다.

In [23]:
_df = df.sort_values('area')
prev_ind = 0
for i in range(0, 10):
    ind = int((i+1)*(23144/10))
    if ind >= 23143:
        ind = None
    df.loc[_df.iloc[prev_ind:ind].index, 'areaRng_id'] = i
    prev_ind = ind

df['areaRng_id'] = df['areaRng_id'].apply(int)
# df.loc[df['area'] < 3133.69, 'areaRngs_fold'] = 0
# df.loc[df['area'] 6990.08
# df.loc[df['area'] 13694.67
# df.loc[df['area'] 23418.57
# df.loc[df['area'] 38938.47
# df.loc[df['area'] 62007.84
# df.loc[df['area'] 95748.3
# df.loc[df['area'] 149517.72
# df.loc[df['area'] 251634.04
# df.loc[df['area']  1048371.21

#0 3133.69
#1 6990.08
#2 13694.67
#3 23418.57
#4 38938.47
#5 62007.84
#6 95748.3
#7 149517.72
#8 251634.04
#9 1048371.21

In [24]:
# %%timeit
# 838 ms ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

labels_num = df['category_id'].max() + 1
# https://stackoverflow.com/a/39132900/14019325
# 기존 코드의 첫번째 loop와 동일합니다. 각 image 별 label 개수를 확인합니다.
y_counts_per_group = df.groupby(['image_id', 'category_id']).size().unstack(fill_value=0)
y_counts_per_fold = np.zeros((k, labels_num))

# scale을 미리 계산하여 연산을 줄입니다.
y_norm_counts_per_group = y_counts_per_group / y_counts_per_group.sum()
# suffle & sort
shuffled_and_sorted_index = y_norm_counts_per_group.sample(frac=1, random_state=seed).std(axis=1).sort_values(ascending=False).index
y_norm_counts_per_group = y_norm_counts_per_group.loc[shuffled_and_sorted_index]

groups_per_fold = defaultdict(set)

# Only y_counts_per_fold
for g, y_counts in zip(y_norm_counts_per_group.index, y_norm_counts_per_group.values):
    best_fold = None
    min_eval = None
    for fold_i in range(k):
        y_counts_per_fold[fold_i] += y_counts
        fold_eval = y_counts_per_fold.std(axis=0).mean()  # numpy를 활용하여 연산을 단순화 합니다.
        y_counts_per_fold[fold_i] -= y_counts

        if min_eval is None or fold_eval < min_eval:
            min_eval = fold_eval
            best_fold = fold_i
    y_counts_per_fold[best_fold] += y_counts
    groups_per_fold[best_fold].add(g)


In [33]:

# labels_num = y.max() + 1
labels_num = df['category_id'].max() + 1

catId_counts_per_group = df.groupby(['image_id', 'category_id']).size().unstack(fill_value=0)
areaRng_counts_per_group = df.groupby(['image_id', 'areaRng_id']).size().unstack(fill_value=0)
catId_counts_per_fold = np.zeros((k, labels_num))
areaRng_counts_per_fold = np.zeros((k, labels_num))

# scale을 미리 계산하여 연산을 줄입니다.
catId_norm_counts_per_group = catId_counts_per_group / catId_counts_per_group.sum()
areaRng_norm_counts_per_group = areaRng_counts_per_group / areaRng_counts_per_group.sum()

# suffle & sort
# shuffled_and_sorted_index = catId_norm_counts_per_group.sample(frac=1, random_state=seed).std(axis=1).sort_values(ascending=False).index
norm_catId_and_areaRng_std = catId_norm_counts_per_group.std(axis=1) + areaRng_norm_counts_per_group.std(axis=1)
shuffled_and_sorted_index = norm_catId_and_areaRng_std.sample(frac=1, random_state=seed).sort_values(ascending=False).index

catId_norm_counts_per_group = catId_norm_counts_per_group.loc[shuffled_and_sorted_index]
areaRng_norm_counts_per_group = areaRng_norm_counts_per_group.loc[shuffled_and_sorted_index]

groups_per_fold = defaultdict(set)

for g, catId_counts, areaRng_counts in zip(catId_norm_counts_per_group.index, catId_norm_counts_per_group.values, areaRng_norm_counts_per_group.values):
    best_fold = None
    min_eval = None
    for fold_i in range(k):
        catId_counts_per_fold[fold_i] += catId_counts
        areaRng_counts_per_fold[fold_i] += areaRng_counts
        fold_catIds_eval = catId_counts_per_fold.std(axis=0).mean()
        fold_areaRngs_eval = areaRng_counts_per_fold.std(axis=0).mean()
        catId_counts_per_fold[fold_i] -= catId_counts
        areaRng_counts_per_fold[fold_i] -= areaRng_counts
        fold_eval = fold_catIds_eval + fold_areaRngs_eval
        if min_eval is None or fold_eval < min_eval:
            min_eval = fold_eval
            best_fold = fold_i
    catId_counts_per_fold[best_fold] += catId_counts
    areaRng_counts_per_fold[best_fold] += areaRng_counts
    groups_per_fold[best_fold].add(g)

In [8]:
def get_distribution(y_vals):
        y_distr = Counter(y_vals)
        y_vals_sum = sum(y_distr.values())
        return [f'{y_distr[i] / y_vals_sum:.2%}' for i in range(np.max(y_vals) + 1)]

In [35]:
groups = df['image_id']

all_groups = set(groups)

distrs = [get_distribution(df['category_id'])]
area_distrs = [get_distribution(df['areaRng_id'])]
index = ['training set']

for i in range(k):
    train_groups = all_groups - groups_per_fold[i]
    test_groups = groups_per_fold[i]

    train_df = df.loc[df['image_id'].isin(train_groups)]
    valid_df = df.loc[df['image_id'].isin(test_groups)]
    # df.loc[df['image_id'].isin(test_groups), 'catIds_fold'] = i

    distrs.append(get_distribution(train_df['category_id'].values))
    index.append(f'train set - fold {i + 1}')
    distrs.append(get_distribution(valid_df['category_id'].values))
    index.append(f'validation set - fold {i + 1}')

    area_distrs.append(get_distribution(train_df['areaRng_id'].values))
    area_distrs.append(get_distribution(valid_df['areaRng_id'].values))

print('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(labels_num)])

Distribution per class:


,Label 0,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7,Label 8,Label 9
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
train set - fold 1,17.14%,27.45%,3.88%,4.05%,4.24%,12.72%,5.45%,22.38%,0.67%,2.02%
validation set - fold 1,17.12%,27.38%,3.84%,4.01%,4.27%,12.72%,5.48%,22.29%,0.86%,2.03%
train set - fold 2,17.14%,27.45%,3.87%,4.04%,4.24%,12.71%,5.46%,22.37%,0.69%,2.02%
validation set - fold 2,17.13%,27.41%,3.88%,4.06%,4.27%,12.73%,5.44%,22.36%,0.69%,2.03%
train set - fold 3,17.13%,27.44%,3.87%,4.05%,4.24%,12.72%,5.46%,22.38%,0.69%,2.02%
validation set - fold 3,17.16%,27.45%,3.89%,4.02%,4.24%,12.71%,5.45%,22.35%,0.69%,2.03%
train set - fold 4,17.13%,27.44%,3.88%,4.05%,4.24%,12.72%,5.45%,22.37%,0.69%,2.02%
validation set - fold 4,17.16%,27.48%,3.85%,4.02%,4.24%,12.71%,5.49%,22.39%,0.65%,2.03%
train set - fold 5,17.14%,27.44%,3.87%,4.05%,4.24%,12.72%,5.45%,22.37%,0.69%,2.02%


In [36]:
pd.DataFrame(area_distrs, index=index, columns=[f'Label {l}' for l in range(labels_num)])

,Label 0,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7,Label 8,Label 9
training set,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
train set - fold 1,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.01%
validation set - fold 1,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,9.96%
train set - fold 2,10.00%,10.00%,10.00%,10.00%,10.01%,10.00%,10.00%,10.00%,10.00%,10.00%
validation set - fold 2,10.01%,10.01%,10.01%,9.97%,9.97%,9.97%,10.01%,10.01%,10.01%,10.01%
train set - fold 3,9.99%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
validation set - fold 3,10.03%,9.99%,9.99%,9.99%,10.03%,9.99%,9.99%,9.99%,9.99%,10.03%
train set - fold 4,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
validation set - fold 4,9.98%,9.98%,10.03%,10.03%,10.03%,9.98%,9.98%,9.98%,9.98%,10.03%
train set - fold 5,10.00%,9.99%,10.00%,10.00%,10.00%,9.99%,10.00%,10.00%,10.00%,10.00%


In [ ]:
# df['catIds_fold'] = df['catIds_fold'].apply(int)
df.to_csv('stratified.10fold.csv', index=False)
df

In [ ]:

all_groups = set(groups)

distrs = [get_distribution(y)]
index = ['training set']

for i in range(k):
    train_groups = all_groups - groups_per_fold[i]
    test_groups = groups_per_fold[i]

    train_df = df.loc[df['image_id'].isin(train_groups)]
    valid_df = df.loc[df['image_id'].isin(test_groups)]

    distrs.append(get_distribution(train_df['category_id'].values))
    index.append(f'train set - fold {i + 1}')
    distrs.append(get_distribution(valid_df['category_id'].values))
    index.append(f'validation set - fold {i + 1}')

print('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(np.max(y) + 1)])

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=100)
train_df['category_id'].hist(ax=ax, bins=np.arange(11)-0.4, width=0.8, label='train')
valid_df['category_id'].hist(ax=ax, bins=np.arange(11)-0.4, width=0.8, label='valid')
ax.set_xticks([_ for _ in range(10)])
# ax.set_xlim(-1, 11)
ax.set_xlabel('category', fontdict={'size': 12})
ax.grid(False)
ax.legend()
plt.tight_layout()

# plt.savefig("train_valid_distribution.png")

In [37]:
train_df.to_csv('stratified_train.10fold.wArea.csv', index=False)
valid_df.to_csv('stratified_valid.10fold.wArea.csv', index=False)
# df.to_csv('annotations.csv', index=False)